In [1]:
import sys
sys.path.append('../')
# import necessary libraries
import numpy as np
import cvxpy as cp
import pandas as pd
from utils.synth import *
from sklearn.linear_model import Ridge

In [2]:
# hyper-parameters
alpha = 0.2
lam = 0.1
threshold = 1e-4

# Paths to the files
path_X0_unscaled = './X0_unscaled.txt'
path_X0 = './X0.txt'
path_X1 = './X1.txt'
path_Y0 = './Y0.txt'
path_Y1 = './Y1.txt'

# Reading each file into a separate DataFrame
X0_unscaled = pd.read_csv(path_X0_unscaled, delim_whitespace=True)
X0 = pd.read_csv(path_X0, delim_whitespace=True)
X1 = pd.read_csv(path_X1, delim_whitespace=True)
Y0 = pd.read_csv(path_Y0, delim_whitespace=True)
Y1 = pd.read_csv(path_Y1, delim_whitespace=True)
Y1 = Y1.to_numpy().flatten()
Y0 = Y0.to_numpy().flatten()


n1 = len(X1)
n0 = len(X0)
p = X0.shape[1]

In [3]:
# L infinity norm approach
lam = loo_lambda_parallel(Y0, X0, norm='L-infty', verbose=False)
print(lam)
Wln = solve_w_parallel(X1, X0, lam, norm='L-infty') # 0.1 works fine
# print('finished 2')
# Wnp = solve_w_parallel(X1, X0, lam, method='nonpenalized')
# print('finished 3')
# Wpure = solve_w_parallel(X1, X0, lam, Wnp=Wnp, method='pure')
# print('finished 4')

/home/youhui/.local/lib/python3.10/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/youhui/.local/lib/python3.10/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/youhui/.local/lib/python3.10/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/youhui/.local/lib/python3.10/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/youhui/.local/lib/python3.10/site-pack

SolverError: Solver 'ECOS' failed. Try another solver, or solve with verbose=True for more information.

In [ ]:
# import time
# from scipy.spatial import Delaunay
# # Start timing Delaunay triangulation
# start_delaunay = time.time()
# DT = Delaunay(sampled_X0)
# end_delaunay = time.time()

# # Timing for finding simplices
# start_simplicies = time.time()
# simplicies = DT.simplices
# end_simplicies = time.time()

# # Timing for finding simplex of each point in X1
# start_find_simplex = time.time()
# tris = DT.find_simplex(sampled_X1)
# end_find_simplex = time.time()

# # Timing for the loop
# start_loop = time.time()
# W = np.zeros_like(Wnp)

# # Initialize accumulators for time measurements
# time_temp_calculation = 0
# time_coord_calculation = 0
# time_writing_W = 0

# for i, point in enumerate(sampled_X1):
#     start_loop_iteration = time.time()

#     tri = tris[i]  # Find the simplex containing X_i
#     if tri != -1:
#         # Time temp calculation
#         start_temp = time.time()
#         temp = DT.transform[tri,:p].dot((point - DT.transform[tri,p]).transpose())
#         end_temp = time.time()
#         time_temp_calculation += end_temp - start_temp

#         # Time coord calculation
#         start_coord = time.time()
#         coord = np.append(temp, 1 - temp.sum())
#         end_coord = time.time()
#         time_coord_calculation += end_coord - start_coord

#         # Time writing W
#         start_writing_W = time.time()
#         idx = simplicies[tri,:]
#         W[idx,i] = coord
#         end_writing_W = time.time()
#         time_writing_W += end_writing_W - start_writing_W
#     else:
#         # Time writing W for else condition
#         start_writing_W_else = time.time()
#         W[:,i] = Wnp[:,i]
#         end_writing_W_else = time.time()
#         time_writing_W += end_writing_W_else - start_writing_W_else

# # Calculate average times
# avg_temp_calculation = time_temp_calculation / len(sampled_X1)
# avg_coord_calculation = time_coord_calculation / len(sampled_X1)
# avg_writing_W = time_writing_W / len(sampled_X1)

# end_loop = time.time()

# # Print the timings
# print(f"Delaunay triangulation time: {end_delaunay - start_delaunay} seconds")
# print(f"Finding simplices time: {end_simplicies - start_simplicies} seconds")
# print(f"Finding simplex for each point time: {end_find_simplex - start_find_simplex} seconds")
# print(f"Loop execution time: {end_loop - start_loop} seconds")
# # Print the average timings
# print(f"Average time for temp calculation per iteration: {avg_temp_calculation} seconds")
# print(f"Average time for coord calculation per iteration: {avg_coord_calculation} seconds")
# print(f"Average time for writing W per iteration: {avg_writing_W} seconds")

In [ ]:
# sampled_Y1 = Y1.loc[sampled_indices1]
# sampled_Y0 = Y0.loc[sampled_indices0]
# sampled_Y1 = sampled_Y1.iloc[:, 0].values
# sampled_Y0 = sampled_Y0.iloc[:, 0].values
# estimate treatment effects
# tau_ln = sampled_Y1 - sampled_Y0 @ Wln
# tau_p = sampled_Y1 - sampled_Y0 @ Wp
# tau_np = sampled_Y1 - sampled_Y0 @ Wnp
# tau_pure = sampled_Y1 - sampled_Y0 @ Wpure

tau_ln = Y1 - Y0 @ Wln
# tau_p = Y1 - Y0 @ Wp
# tau_np = Y1 - Y0 @ Wnp
# tau_pure = Y1 - Y0 @ Wpure
# print(tau_p)
# print(tau_np)

# Treatment effects
tt_ln = np.mean(tau_ln)
# tt_p = np.mean(tau_p)
# tt_np = np.mean(tau_np)
# tt_pure = np.mean(tau_pure)
# print(rmse_aggreg)
# density
dens_ln = np.count_nonzero(Wln > threshold) / n1
# dens_p = np.count_nonzero(Wp > threshold) / n1
# dens_np = np.count_nonzero(Wnp > threshold) / n1
# dens_pure = np.count_nonzero(Wpure > threshold) / n1
# print(density)
# magnitude
mag_ln = Wln.max(axis=0).mean()
# mag_p = Wp.max(axis=0).mean()
# mag_np = Wnp.max(axis=0).mean()
# mag_pure = Wpure.max(axis=0).mean()

In [ ]:
ridge_model = Ridge(alpha=alpha) # Adjust alpha as needed
ridge_model.fit(X0, Y0)
Y1_corr = Y1 - ridge_model.predict(X1)
Y0_corr = Y0 - ridge_model.predict(X0)
# ridge_model.fit(sampled_X0, sampled_Y0)
# Y1_corr = sampled_Y1 - ridge_model.predict(sampled_X1)
# Y0_corr = sampled_Y0 - ridge_model.predict(sampled_X0)
# estimate treatment effects
tau_ln_bc = Y1_corr - Y0_corr @ Wln
# tau_p_bc = Y1_corr - Y0_corr @ Wp
# tau_np_bc = Y1_corr - Y0_corr @ Wnp
# tau_pure_bc = Y1_corr - Y0_corr @ Wpure
# print(tau_p)
# print(tau_np)

# summarize the results
# Treatment effects
tt_ln_bc = np.mean(tau_ln_bc)
# tt_p_bc = np.mean(tau_p_bc)
# tt_np_bc = np.mean(tau_np_bc)
# tt_pure_bc = np.mean(tau_pure_bc)

In [ ]:
import pandas as pd

# Define the data for the matrix
data = [[tt_ln, dens_ln, mag_ln], 
        # [tt_p, dens_p, mag_p],
        # [tt_np, dens_np, mag_np],
        # [tt_pure, dens_pure, mag_pure],
        [tt_ln_bc, None, None]]
        # [tt_p_bc, None, None],
        # [tt_np_bc, None, None],]
        # [tt_pure_bc, None, None]]

# Define row and column names
# methods = ['L-infinity', 'Penalized', 'Nonpenalized', #'Pure',
#            'L-infinity-bc', 'Penalized-bc', 'Nonpenalized-bc']#, 'Pure-bc']
# metrics = ['Treatment Effects', 'Density', 'Magnitude']
methods = ['L-infty', 
           'L-infty-bc']
metrics = ['Treatment Effects', 'Density', 'Magnitude']

# Create a DataFrame
df = pd.DataFrame(data, index=methods, columns=metrics)

print(df)

In [ ]:
latex_table = df.to_latex(index=True)

# Print or write the LaTeX table to a file
print(latex_table)